In [1]:
import sqlite3
con = sqlite3.connect("movies.db")
cur = con.cursor()

#### Country


In [2]:
from enum import Enum
class Country(Enum):
    USA = 'USA'
    UKRAINE = 'Ukraine'
    ITALY = 'Italy'
    FRANCE = 'France'

    @classmethod
    def valueTuples(self):
        return [(e.value,) for e in Country]

    @classmethod
    def createTable(self):
        cur.execute('CREATE TABLE IF NOT EXISTS l_countries (country TEXT PRIMARY KEY)')
        cur.executemany('INSERT INTO l_countries VALUES(?)', Country.valueTuples())
        con.commit()

    @classmethod
    def removeTable(self):
        cur.execute('DROP TABLE IF EXISTS l_countries')
        con.commit()
        
    @classmethod
    def getAll(self):
        cur.execute('SELECT * FROM l_countries')
        sqlResult = cur.fetchall()
        return [Country(p[0]) for p in sqlResult]

In [3]:
Country.removeTable()
Country.createTable()

In [4]:
Country.getAll()

[<Country.USA: 'USA'>,
 <Country.UKRAINE: 'Ukraine'>,
 <Country.ITALY: 'Italy'>,
 <Country.FRANCE: 'France'>]

#### Genre

In [5]:
class Genre(Enum):
    FICTION = 'fiction'
    DRAMA = 'drama'
    ACTION = 'action'
    BIOPIC = 'biopic'
    OTHERS = 'others'
  

    @classmethod
    def valueTuples(self):
        return [(e.value,) for e in Genre]

    @classmethod
    def createTable(self):
        cur.execute('CREATE TABLE IF NOT EXISTS l_genres (genre text)')
        cur.executemany('INSERT INTO l_genres VALUES(?)', Genre.valueTuples())
        con.commit()

    @classmethod
    def removeTable(self):
        cur.execute('DROP TABLE IF EXISTS l_genres')
        con.commit()
        
    @classmethod
    def getAll(self):
        cur.execute('SELECT * FROM l_genres')
        sqlResult = cur.fetchall()
        return [Genre(p[0]) for p in sqlResult]

In [6]:
Genre.removeTable()
Genre.createTable()
Genre.getAll()

[<Genre.FICTION: 'fiction'>,
 <Genre.DRAMA: 'drama'>,
 <Genre.ACTION: 'action'>,
 <Genre.BIOPIC: 'biopic'>,
 <Genre.OTHERS: 'others'>]

In [7]:
def createGenreMapping():
    cur.execute('''CREATE TABLE IF NOT EXISTS MOVIE_GENRE(
                    movie text,
                    genre text,
                    PRIMARY KEY(movie, genre),
                    FOREIGN KEY(movie) REFERENCES movies(name),
                    FOREIGN KEY(genre) REFERENCES l_genres(genre))''')
    con.commit()

In [8]:
cur.execute('DROP TABLE IF EXISTS MOVIE_GENRE')
con.commit()

In [9]:
createGenreMapping()

#### Movie

In [10]:
class Movie:
    def __init__(self, name, director, country, year, rating, genres):
        self.name = name
        self.director = director
        self.country = country
        self.year = year
        self.rating = rating
        self.genres = genres

    @classmethod
    def createTable(self):
        cur.execute('''CREATE TABLE IF NOT EXISTS movies (
                        name text PRIMARY KEY,
                        director text,
                        country text,
                        year integer,
                        rating REAL,
                        FOREIGN KEY(country) REFERENCES l_countries(country))''')
        con.commit()

    def insert(self):
        cur.execute('''INSERT INTO movies VALUES(?,?,?,?, ?)''', (self.name, self.director, self.country, self.year, self.rating))
        cur.executemany('''INSERT INTO movie_genre VALUES(?,?)''', [(self.name, m.value) for m in self.genres])
        con.commit()

    def update(self):
        cur.execute('''UPDATE movies SET director=?, country=?, year=?, rating=? WHERE name=?''', (self.director, self.country, self.year, self.rating, self.name))
        con.commit()
        
    @classmethod
    def removeTable(self):
        cur.execute('DROP TABLE IF EXISTS movies')
        con.commit()
        
    @classmethod
    def get(self, name):
        cur.execute(''' SELECT director, country, year, rating, GROUP_CONCAT(genre, ',') FROM movies
                    LEFT JOIN movie_genre ON movies.name = movie_genre.movie
                    WHERE movies.name=? GROUP BY movies.name''', (name,))
        res = cur.fetchone()
        if res:
            return Movie(name, res[0], res[1], res[2], res[3], getGenresFromString(res[4]))
        else:
            return None

    def __str__(self):
        return "(%s, %s, %s, %s, %s, %s)" % (self.name, self.director, self.country, self.year, self.rating, self.genres)

In [11]:
def getGenresFromString(str):
    return {Genre(e) for e in str.split(',')}

In [12]:
Movie.removeTable()

In [13]:

Movie.createTable()

In [14]:
movie1 = Movie('It Chapter One', 'Andy Muschietti', 'USA', 2017, 7.3, {Genre.OTHERS})
movie2 = Movie('Interstellar', 'Christopher Nolan', 'USA', 2014, 8.7, {Genre.FICTION, Genre.ACTION})

movie1.insert()
movie2.insert()

In [15]:
print(Movie.get('Interstellar'))

(Interstellar, Christopher Nolan, USA, 2014, 8.7, {<Genre.ACTION: 'action'>, <Genre.FICTION: 'fiction'>})


#### Director

In [16]:
class Director:
    def __init__(self, name, birth_year, country):
        self.name = name
        self.birth_year = birth_year
        self.country = country

    @classmethod
    def createTable(cls):
        cur.execute('''CREATE TABLE IF NOT EXISTS directors (
                        name TEXT PRIMARY KEY,
                        birth_year INTEGER,
                        country TEXT)''')
        con.commit()

    @classmethod
    def removeTable(cls):
        cur.execute('DROP TABLE IF EXISTS directors')
        con.commit()

    def insert(self):
        cur.execute('''INSERT INTO directors VALUES(?,?,?)''', (self.name, self.birth_year, self.country))
        con.commit()

    def update(self):
        cur.execute('''UPDATE directors SET birth_year=?, country=? WHERE name=?''', (self.birth_year, self.country, self.name))
        con.commit() 

    @classmethod
    def get(self, name):
        cur.execute('SELECT birth_year, country FROM directors WHERE name=?', (name,))
        res = cur.fetchone()
        if res:
            return Director(name, res[0], res[1])
        else:
            return None

    @classmethod
    def getAll(self):
        cur.execute('SELECT name, birth_year, country FROM directors')
        res = cur.fetchall()
        return [Director(el[0], el[1], el[2]) for el in res]

    def __str__(self):
        return "(%s, %s, %s)" % (self.name, self.birth_year, self.country)


In [17]:
Director.removeTable()
Director.createTable()

In [18]:
d1 = Director('Christopher Nolan',  1970, 'USA')
d2 = Director('Andy Muschietti', 1978, 'Argentine')

d1.insert()
d2.insert()

In [19]:
directorFromDb = Director.get('Christopher Nolan')
print(directorFromDb)

(Christopher Nolan, 1970, USA)


In [52]:
class Movies:
    def __init__(self, name, director, country, year, rating, genres):
        self.name = name
        self.director = director
        self.country = country
        self.year = year
        self.rating = rating
        self.genres = genres

    @classmethod
    def createTable(self):
        cur.execute('''CREATE TABLE IF NOT EXISTS movies (
                        name TEXT PRIMARY KEY,
                        director TEXT,
                        country TEXT,
                        year INTEGER,
                        rating REAL,
                        FOREIGN KEY(director) REFERENCES directors(name))''')
        con.commit()

    def insert(self):
        cur.execute('''INSERT INTO movies VALUES(?,?,?,?,?)''', (self.name, self.director, self.country, self.year, self.rating))
        cur.executemany('''INSERT INTO movie_genre VALUES(?,?)''', [(self.name, genre.value) for genre in self.genres])
        con.commit()

    def update(self):
        cur.execute('''UPDATE movies SET director=?, country=?, year=?, rating=? WHERE name=?''', 
                    (self.director, self.country, self.year, self.rating, self.name))
        con.commit() 


    @classmethod
    def get(self):
        cur.execute('''
                    SELECT m.name, m.director, d.birth_year, d.country, m.country, m.year, m.rating, GROUP_CONCAT(g.genre, ',')
                    FROM movies m
                    LEFT JOIN directors d ON m.director = d.name
                    LEFT JOIN movie_genre g ON m.name = g.movie
                    GROUP BY m.name
                    ''')
        res = cur.fetchall()
        return res
    def __str__(self):
        return "(%s, %s, %s, %s, %s, %s)" % (self.name, self.director, self.country, self.year, self.rating, self.genres)


In [53]:
Movie.removeTable()

In [54]:
Movie.createTable()

In [55]:
cur.execute('DROP TABLE IF EXISTS MOVIE_GENRE')
con.commit()

In [56]:
createGenreMapping()

In [57]:
movie1 = Movies('It Chapter One', 'Andy Muschietti', 'USA', 2017, 7.3, {Genre.OTHERS})
movie2 = Movies('Interstellar', 'Christopher Nolan', 'USA', 2014, 8.7, {Genre.FICTION, Genre.ACTION})

movie1.insert()
movie2.insert()

In [58]:
print(movie1.get())

[('Interstellar', 'Christopher Nolan', 1970, 'USA', 'USA', 2014, 8.7, 'action,fiction'), ('It Chapter One', 'Andy Muschietti', 1978, 'Argentine', 'USA', 2017, 7.3, 'others')]


In [59]:
movies = movie2.get()
formatted_movies = []
for movie in movies:
    formatted_movies.append(str(movie))
print(formatted_movies)

["('Interstellar', 'Christopher Nolan', 1970, 'USA', 'USA', 2014, 8.7, 'action,fiction')", "('It Chapter One', 'Andy Muschietti', 1978, 'Argentine', 'USA', 2017, 7.3, 'others')"]


Diagram

```mermaid
erDiagram


    movie {
        TEXT name PK
        TEXT director
        TEXT country FK
        INTEGER year
        REAL rating

    }

    l_countries {
        TEXT country PK
    }

    directors {
        TEXT name PK
        INTEGER year
        TEXT country
    }

    l_genres {
        TEXT genre PK
    }

    movie_genres {
        TEXT movie FK
        TEXT genre FK
    }


    movie }|--|| directors : "made by"
    movie }|--|| l_countries : from
    movie }|--|{ movie_genres  : "is"
    l_genres||--|{ movie_genres : refers
```  